## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-18-04-04-54 +0000,nyt,"Venezuela Strongman and Maduro Ally, Diosdado ...",https://www.nytimes.com/2026/01/15/world/ameri...
1,2026-01-18-04-00-00 +0000,wsj,Germany’s Military Recruitment Drive Has a Gen...,https://www.wsj.com/world/europe/germanys-mili...
2,2026-01-18-04-00-00 +0000,wsj,Fed Turmoil Is Threatening Dollar Supremacy Ju...,https://www.wsj.com/finance/currencies/fed-tur...
3,2026-01-18-03-56-44 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...
4,2026-01-18-03-56-31 +0000,nyt,Virginia’s New Governor Moves Swiftly to Overh...,https://www.nytimes.com/2026/01/17/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2400/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,30
26,new,12
286,iran,9
362,leader,8
30,greenland,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2026-01-18-03-56-44 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...,68
100,2026-01-17-17-03-12 +0000,nypost,"Trump slaps increased tariffs on Denmark, Euro...",https://nypost.com/2026/01/17/world-news/how-t...,66
93,2026-01-17-17-54-25 +0000,nypost,Trump calls for ‘new leadership in Iran’: Time...,https://nypost.com/2026/01/17/world-news/trump...,61
42,2026-01-17-23-25-00 +0000,wsj,Trump to Hit European Nations With 10% Tariffs...,https://www.wsj.com/politics/policy/trump-to-h...,60
38,2026-01-17-23-39-58 +0000,wapo,Trump announces tariffs on European countries ...,https://www.washingtonpost.com/politics/2026/0...,57


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,68,2026-01-18-03-56-44 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...
76,33,2026-01-17-20-14-57 +0000,bbc,Iran supreme leader acknowledges thousands kil...,https://www.bbc.com/news/articles/ckglee733wno...
94,30,2026-01-17-17-50-20 +0000,bbc,Blair and Rubio among names on senior executiv...,https://www.bbc.com/news/articles/c07xv92vrz2o...
30,29,2026-01-18-00-41-25 +0000,nypost,Minnesota National Guard on ‘standby’ and reve...,https://nypost.com/2026/01/17/us-news/minnesot...
116,29,2026-01-17-13-50-56 +0000,bbc,He once criticised African leaders who cling t...,https://www.bbc.com/news/articles/cr7jep39eg4o...
63,27,2026-01-17-21-29-53 +0000,nypost,California’s largest hospital group owes patie...,https://nypost.com/2026/01/17/us-news/kaiser-p...
46,26,2026-01-17-22-44-56 +0000,nypost,Venezuelan acting president Delcy Rodriguez wa...,https://nypost.com/2026/01/17/world-news/venez...
53,25,2026-01-17-22-13-12 +0000,nypost,"NYC man allegedly told his mom, ‘I killed gran...",https://nypost.com/2026/01/17/us-news/nyc-man-...
92,24,2026-01-17-17-55-58 +0000,cbc,"European Union, South America's Mercosur bloc ...",https://www.cbc.ca/news/world/trade-agreement-...
22,23,2026-01-18-01-29-47 +0000,nypost,Cops slam ‘activist’ judge putting lives in d...,https://nypost.com/2026/01/17/us-news/police-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
